In [127]:
import serial
import time
from intelhex import IntelHex
from intelhex import AddressOverlapError, HexRecordError

In [129]:
ser = serial.Serial('/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=0, rtscts=0)
print(ser.name)

/dev/ttyUSB1


In [59]:
ser.write(b"Ahoj")

4

In [130]:
def write(b):
    print("WRITE>", b, [hex(x) for x in b])
    ser.write(b)

In [134]:
ser.dtr = True
ser.rts = True
time.sleep(1/20)
ser.dtr = False
ser.rts = False
time.sleep(1/50)


print("GET SYNC")
ser.timeout = 0.2
ser.blocking = True
write(b'0 ')
ser.flush()
for x in range(15):
    print("try", x)
    d = ser.read(2)
    print("REC", [hex(i) for i in d], len(d))
    if not len(d):
        ser.write(b' ')
        continue
    if d == b'\x14\x10':
        print("SYNC OBTAINED")
        break
    
time.sleep(0.1)

GET SYNC
WRITE> b'0 ' ['0x30', '0x20']
try 0
REC ['0x14', '0x10'] 2
SYNC OBTAINED


In [125]:

ser.timeout = 1
print("CHECK BT")
write(b'\x31\x20')
d = ser.read(8)
print("RESPONSE", [hex(i) for i in d] )
    

CHECK BT
WRITE> b'1 ' ['0x31', '0x20']
RESPONSE ['0xa0', '0x8d']


In [126]:

print("SET PROGRAMMING MODE")
ser.write(b'P ')
d = ser.read(2)
print("RESPONSE", [hex(i) for i in d] )
    

SET PROGRAMMING MODE
RESPONSE ['0xa0', '0x8d']


In [60]:

    
# print("GET PARAM")
# ser.write(b'A\x80 ')
# for x in range(15):
#     d = ser.read(2)
#     print(d)
#     if len(d): break
#     ser.write(b' ')
    
    
    
ih = IntelHex()
ih.fromfile('/tmp/arduino/sketches/7115AB02B25FC6E7C7762E92029F1EF6/Blink.ino.hex', format='hex')
page = 256
#page = 2

for address in range(0, ih.maxaddr(), page):
    print("BUFFER", address)
    buffer = bytearray(ih.tobinarray(start=address, size=page))
    print("BUF,", buffer)
    buff_len = len(buffer)
    cmd = bytearray(4)
    cmd[0] = ord('d')
    cmd[1] = ((buff_len >> 8) & 0xFF)
    cmd[2] = (buff_len & 0xFF)
    cmd[3] = ord('F')

    cmd.extend(buffer)
    cmd.append(ord(' '))
    
    print("PRIKAZ", cmd)
    ser.write(cmd)
    
    for x in range(15):
        d = ser.read(2)
        print(d)
        if len(d): break
# for address in range(0, ih.maxaddr(), page):
#     buffer = ih.tobinarray(start=address, size=page)
#     buffer = bytearray(buffer)
#     payload = bytearray(b'\x60') + buffer + bytearray(b' ')
#     print("Offset",address,"Payload", payload)
#     ser.write(payload)
#     # time.sleep(0.01)

#     for x in range(15):
#         d = ser.read(2)
#         print(d)
#         if len(d): break
#         ser.write(b' ')

GET SYNC
b''
SET PROGRAMMING MODE
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
BUFFER 0
BUF, bytearray(b'\x0c\x94\x80\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94F\x01\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x0c\x94\x92\x00\x00\x00!\x00$\x00\'\x00*\x00\x00\x00"\x00%\x00(\x00+\x00\x02\x02\x02\x02\x02\x02\x02\x02\x04\x04\x04\x04\x04\x04\x04\x04\x03\x03\x03\x03\x03\x03\x03\x03\x01\x01\x01\x01\x01\x01\x01\x01\x01\x02\x04\x08\x10 @\x80\x01\x02\x04\x08\x10 @\x80\x01\x02\x04\x08\x10 @\x80\x01\x02\x04\x08\x10 @\x80\x00\x00\x00\x02\x03\x00\n\x0b\x00

KeyboardInterrupt: 